IMPORT LIBRARIES

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 112kB 17.0MB/s eta 0:00:01
Folium installed
Libraries imported.


FOURSQUARE CREDENTIAL AND VERSION

In [2]:
CLIENT_ID = 'KZF3XOP2LUEBHCWEIXZ5IHQUCICYJZ2ZI3EIGUV5XX5BXQ3F' # your Foursquare ID
CLIENT_SECRET = 'RYH542BE11TJCRUA1VJAAIN01BUWWEWFITZA44T0VZFNI3YZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KZF3XOP2LUEBHCWEIXZ5IHQUCICYJZ2ZI3EIGUV5XX5BXQ3F
CLIENT_SECRET:RYH542BE11TJCRUA1VJAAIN01BUWWEWFITZA44T0VZFNI3YZ


CONVERT ADDRESS TO LATTITUE AND LONGTITUDE

In [3]:
address = 'Austin, TX'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

30.2711286 -97.7436995


SEARCH QUERY FOR jobs

In [32]:
search_query = 'Jobs'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=KZF3XOP2LUEBHCWEIXZ5IHQUCICYJZ2ZI3EIGUV5XX5BXQ3F&client_secret=RYH542BE11TJCRUA1VJAAIN01BUWWEWFITZA44T0VZFNI3YZ&ll=30.2711286,-97.7436995&v=20180604&query=Jobs&radius=500&limit=30'

SEND THE GET REQUEST AND EXAMINE THE RESULTS

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc54ce8b144236e12908cb8'},
 'response': {'venues': [{'id': '5728f9d4498e4d039ce99626',
    'name': 'ServJobs',
    'location': {'lat': 30.266925384556114,
     'lng': -97.74338722229004,
     'labeledLatLngs': [{'label': 'display',
       'lat': 30.266925384556114,
       'lng': -97.74338722229004}],
     'distance': 468,
     'cc': 'US',
     'city': 'Austin',
     'state': 'TX',
     'country': 'United States',
     'formattedAddress': ['Austin, TX', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ff941735',
      'name': 'Miscellaneous Shop',
      'pluralName': 'Miscellaneous Shops',
      'shortName': 'Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1606765800',
    'hasPerk': False}]}}

GET VENUES FROM JSON AND TRANSFORM IT INTO PANDAS DATAFRAME

In [34]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress
0,5728f9d4498e4d039ce99626,ServJobs,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1606765800,False,30.266925,-97.743387,"[{'label': 'display', 'lat': 30.26692538455611...",468,US,Austin,TX,United States,"[Austin, TX, United States]"


DEFINE INFO OF INTEREST AND FILTER DATAFRAME

In [35]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,id
0,ServJobs,Miscellaneous Shop,30.266925,-97.743387,"[{'label': 'display', 'lat': 30.26692538455611...",468,US,Austin,TX,United States,"[Austin, TX, United States]",5728f9d4498e4d039ce99626


VISUALIZE INTERSTING PLACES NEARBY

In [36]:
dataframe_filtered.name
#dataframe_filtered.categories

0    ServJobs
Name: name, dtype: object

In [37]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='AUSTIN',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

EXPLORE A GIVEN VENUE

In [16]:
venue_id = '5728f9d4498e4d039ce99626'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5728f9d4498e4d039ce99626?client_id=KZF3XOP2LUEBHCWEIXZ5IHQUCICYJZ2ZI3EIGUV5XX5BXQ3F&client_secret=RYH542BE11TJCRUA1VJAAIN01BUWWEWFITZA44T0VZFNI3YZ&v=20180604'

SEND GET REQUEST FOR RESULT

In [17]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
tip1=result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes'])


EXPLORE A LOCATION

In [18]:
#define a url
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=KZF3XOP2LUEBHCWEIXZ5IHQUCICYJZ2ZI3EIGUV5XX5BXQ3F&client_secret=RYH542BE11TJCRUA1VJAAIN01BUWWEWFITZA44T0VZFNI3YZ&ll=30.2711286,-97.7436995&v=20180604&radius=500&limit=30'

In [20]:
results = requests.get(url).json()
'There are {} around ServJobs.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around ServJobs.'

GET ITEMS FROM JSON

In [21]:
items = results['response']['groups'][0]['items']
items

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4aa06c38f964a520923f20e3',
   'name': "Perry's Steakhouse",
   'location': {'address': '114 W 7th St',
    'crossStreet': 'at Colorado St',
    'lat': 30.269373758193904,
    'lng': -97.74367595773867,
    'labeledLatLngs': [{'label': 'display',
      'lat': 30.269373758193904,
      'lng': -97.74367595773867}],
    'distance': 195,
    'postalCode': '78701',
    'cc': 'US',
    'city': 'Austin',
    'state': 'TX',
    'country': 'United States',
    'formattedAddress': ['114 W 7th St (at Colorado St)',
     'Austin, TX 78701',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1cc941735',
     'name': 'Steakhouse',
     'pluralName': 'Steakhouses',
     'shortName': 'Steakhouse',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/steakhouse_',
      'suffix': '.png'},
     'primary': True}

PROCESS JSON AND CONVERT IT TO A CLEAN DATAFRAME

In [22]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Perry's Steakhouse,Steakhouse,114 W 7th St,at Colorado St,30.269374,-97.743676,"[{'label': 'display', 'lat': 30.26937375819390...",195,78701,US,Austin,TX,United States,"[114 W 7th St (at Colorado St), Austin, TX 787...",NaN,4aa06c38f964a520923f20e3
1,Paramount Theatre,Movie Theater,713 Congress Ave,7th St.,30.269457,-97.742077,"[{'label': 'display', 'lat': 30.26945691311456...",242,78701,US,Austin,TX,United States,"[713 Congress Ave (7th St.), Austin, TX 78701,...",NaN,44b75631f964a5206d351fe3
2,Caffé Medici,Coffee Shop,804 Congress Ave #101,8th Street,30.270119,-97.742154,"[{'label': 'display', 'lat': 30.27011945412414...",186,78701,US,Austin,TX,United States,"[804 Congress Ave #101 (8th Street), Austin, T...",NaN,575825ea498e6d816e1142cb
3,The Townsend,Lounge,718 Congress Ave Ste 100,8th St,30.269611,-97.742448,"[{'label': 'display', 'lat': 30.269611, 'lng':...",207,78701,US,Austin,TX,United States,"[718 Congress Ave Ste 100 (8th St), Austin, TX...",NaN,54f12e4e498e53d73df2472b
4,Upstairs at Caroline,Hotel,621 Congress Ave Ste 201,NaN,30.268810,-97.742310,"[{'label': 'display', 'lat': 30.26881041728249...",290,78701,US,Austin,TX,United States,"[621 Congress Ave Ste 201, Austin, TX 78701, U...",Downtown Austin,59704c66e075506cb46a616a
5,The Iron Bear,Gay Bar,121 W 8th St,at Colorado St,30.270186,-97.743386,"[{'label': 'display', 'lat': 30.27018609829009...",109,78701,US,Austin,TX,United States,"[121 W 8th St (at Colorado St), Austin, TX 787...",Downtown Austin,4ce444f1637c6ea85e6e5742
6,Royal Blue Grocery,Grocery Store,609 Congress Ave,btw 6th St. & 7th St.,30.268378,-97.742678,"[{'label': 'display', 'lat': 30.26837800323493...",321,78701,US,Austin,TX,United States,"[609 Congress Ave (btw 6th St. & 7th St.), Aus...",NaN,4d21da6dd7b0b1f7c980209f
7,CU29 Cocktail Bar,Cocktail Bar,720 Brazos St Ste B-710,Brazos St Between 7th & 8th,30.269133,-97.741159,"[{'label': 'display', 'lat': 30.26913266105620...",330,78701,US,Austin,TX,United States,[720 Brazos St Ste B-710 (Brazos St Between 7t...,NaN,51afdf5a498e237391d3259f
8,The Driskill Bar,Hotel Bar,604 Brazos St,6th St,30.268605,-97.741573,"[{'label': 'display', 'lat': 30.26860536980661...",347,78701,US,Austin,TX,United States,"[604 Brazos St (6th St), Austin, TX 78701, Uni...",NaN,4ba2fdc6f964a520b02638e3
9,Texas State Capitol,Capitol Building,112 E 11th St,at Congress Ave,30.273659,-97.740814,NaN,395,78701,US,Austin,TX,United States,"[112 E 11th St (at Congress Ave), Austin, TX 7...",NaN,49bd2c3af964a52052541fe3


LET'S VISUALIZE ITEMS ON THE MAP

In [24]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='SERVJOBS',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

EXPLORE TRENDING VENUES

In [25]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc547dfadbbaf055559c583'},
 'response': {'venues': []}}

CHECK IF THERE ARE ANY TRENDING VENUES AT THIS TIME

In [26]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [27]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [28]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [29]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'